In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv("market_log.csv")
df.columns = ["timestamp", "crypto_name", "price", "market_cap", "volume", "change_24h"]
print(df.head())


                    timestamp  crypto_name        price    market_cap  \
0  2025-11-05T19:05:03.104179     ethereum  3433.580000  414177024552   
1  2025-11-05T19:05:03.104179       tether     0.999994  183459395045   
2  2025-11-05T19:05:03.104179       ripple     2.280000  137011113796   
3  2025-11-05T19:05:03.104179  binancecoin   959.510000  132159983951   
4  2025-11-05T19:05:03.104179       solana   161.910000   89465311860   

         volume  change_24h  
0   63949503832     1.93734  
1  182314675744     0.04212  
2    8144500431     2.89487  
3    4363493719     3.71528  
4   10138780392     4.05171  


In [14]:
tweets_df = pd.read_csv("tweets_log.csv")
tweets_df.columns = ["timestamp", "crypto_name", "tweet_created_at", "text", "sentiment", "like_count", "retweet_count"]
print(tweets_df.head())

                    timestamp crypto_name tweet_created_at  \
0  2025-11-05T19:05:39.442368    ethereum              10s   
1  2025-11-05T19:05:39.442368    ethereum              10s   
2  2025-11-05T19:05:39.442368    ethereum              11s   
3  2025-11-05T19:05:39.442368    ethereum              12s   
4  2025-11-05T19:05:39.442368    ethereum              14s   

                                                text sentiment  like_count  \
0                                            Amen 🙏📖   neutral         NaN   
1  2/2 Explore how Drosera works, why its archite...  positive         NaN   
2  ETH/USD\n𝐃𝐚𝐭𝐚 𝐝𝐨𝐞𝐬𝐧’𝐭 𝐬𝐥𝐞𝐞𝐩. 𝐍𝐞𝐢𝐭𝐡𝐞𝐫 𝐝𝐨𝐞𝐬 𝐒𝐢𝐛𝐲...  positive         NaN   
3  espresso makes ethereum rollups fast, synced, ...  positive         NaN   
4               All hail King @RAILGUN_Project $RAIL   neutral         NaN   

   retweet_count  
0            NaN  
1            NaN  
2            NaN  
3            NaN  
4            NaN  
